<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="1000" alt="cognitiveclass.ai logo"  />
</center>

# Instructional Lab:  Create a Db2 asset in your IBM Watson Studio project and load data into the database

Estimated time needed: 45 minutes

# Introduction

Using this R notebook you will complete the following tasks:

1.  Create a Db2 'connection asset' in your IBM Watson Studio project
2.  Create a token for your project
3.  Insert the project token into your Jupyter notebook
4.  Use the 'project-Lib' library to fetch your Db2 connection credentials
5.  Establish your Db2 connection
6.  Load the datasets into Db2

You will not be graded in this instructional lab.\
However, you will be asked to query the Db2 tables you create here in the next lab, which contains graded tasks.


Let's begin by going through steps 1-5 for enabling a connection to Db2 from a Jupyter Notebook running an R kernel within your IBM Watson Studio Project.

## Step 1. Create a Db2 'connection asset' in your IBM Watson Studio project

Click '**Add to project +**' and select the '**Connection**' asset type:

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module%203/images/add_connection_asset.png" width="1200" alt=""  />

Select '**Db2**' as the type of data source:

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module%203/images/db2_asset.png" width="1200" alt=""  />

This brings up a new window where you can now enter a name for your connection, the database details, and your credentials. Use the same information that you previously used to connect to Db2 in the labs from **Course 3: SQL for Data Science Using R**.\
Now click '**Create**'. Your new connection should be listed under '**Data assets**' on the Assets page for your IBM Watson Studio project.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module%203/images/newDb2connection.png" width="1200" alt=""  />

## Step 2. Create a token for your project

On the settings page for your project, click on '**New Token +**'. Then enter a Token name and set the access role to '**Editor**', then click '**Create**' <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module%203/images/NewToken.png" width="1200" alt=""  />

## Step 3. Insert the project token into your Jupyter notebook

Click on the icon above your notebook with the three dots, then click 'Insert project token'. This will add a cell to the top of your notebook that you will need to run to enable access to your IBM Watson Studio project resources, and in particular, your Db2 connection. The resulting automatically generated cell should look like this:

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module%203/images/InsertProjectToken.png" width="1200" alt=""  />


## Step 4. Use the 'project-Lib' library to fetch connection credentials

The projectLib library should now be loaded as a result of running the above auto-generated project token cell. Also, the 'project' variable contains everything you need to connect to Db2 from R. Now run the following code to get your connection credentials:


In [ ]:
# Fetch connection
conn.cred <- project$get_connection(name="Db2Connection") # use your connection name in place of "Db2Connection"
conn.cred

## Step 5. Establish your Db2 connection

Load the 'RODBC' library and use the 'odbcConnect( )' function to establish the connection. Once you run the following cell (as is), you will be ready to start running SQL queries using the RODBC library as you did in Course 3.


In [ ]:
library(RODBC)
props <- paste("DASHDB;DATABASE=BLUDB;HOSTNAME=", conn.cred$host, ";PORT=50000;PROTOCOL=TCPIP;", sep="")
conn <- odbcConnect(props, uid = conn.cred$username, pwd = conn.cred$password)

In [ ]:
# Dump connection info
##############################################################
sql.info <- sqlTypeInfo(conn)
conn.info <- odbcGetInfo(conn)
conn.info["DBMS_Name"]
conn.info["DBMS_Ver"]
conn.info["Driver_ODBC_Ver"]

## Step 6. Load the Datasets into Db2

Now let's prepare to load the datasets into Db2 tables.

### Dataset URLs

Use the following urls to load your datasets into R dataframes. The datasets are already clean, but you will still need to pay careful attention to data types, especially dates, which you may need to coerce. Also, ensure any categorical variables get typed as factors.

```RR
seoul_bike_sharing <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/seoul_bike_sharing.csv"  

cities_weather_forecast <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/cities_weather_forecast.csv"  

bike_sharing_systems <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/bike_sharing_systems.csv"  

world_cities <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/world_cities.csv"
```

Name the tables as follows:

### Table Names

1.  SEOUL_BIKE_SHARING
2.  CITIES_WEATHER_FORECAST
3.  BIKE_SHARING_SYSTEMS
4.  WORLD_CITIES


### Check whether the tables already exist, and drop them if so.


In [ ]:
tables <- c("SEOUL_BIKE_SHARING", "CITIES_WEATHER_FORECAST",
            "BIKE_SHARING_SYSTEMS", "WORLD_CITIES") 

for (table in tables) {
    # Drop tables if they already exist
    out <- sqlTables(conn, tableType = "TABLE",
                      tableName = table)
    if (nrow(out)>0) {
        err <- sqlDrop(conn, table,
                        errors=FALSE)  
        if (err==-1) {
            cat("An error has occurred.\n")
            err.msg <- odbcGetErrMsg(conn)
            for (error in err.msg) { 
                cat(error,"\n")
            }
        } 
        else {
            cat ("Table: ",table," was dropped\n")
        }
    }
    else {
          cat ("Table: ", table," does not exist\n")
    }
}


### Dataset reading and table loading considerations

Now you will read each dataset into an R dataframe using the urls provided above, and finally, load your tables.

Instead of first creating a table and specifying the column names and types as we did in the previous SQL course with SN Labs, the IBM Watson Studio environment allows us the convenience to proceed directly to creating and populating the table simultaneously, using the `sqlSave()` function from the `RODBC` package in R.

We will still set `colClasses=c(Date="character")` when reading data with Date columns into a dataframe, but a couple of things will differ from how we accomplished the data loading in the SN Labs environment:

1.  We will need to cast the Date column of the dataframe as type `Date`
2.  When we load the dataframe into a table using `sqlSave()`, we need to specify `varTypes=c(Date="Date")`

This explicitly ensures that Db2 interprets the `Date` column as a date type, so we can later use built-in Db2 date functions.


## Read the datasets into dataframes


In [ ]:
seoul_bike_sharing <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/seoul_bike_sharing.csv"  
cities_weather_forecast <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/cities_weather_forecast.csv"  
bike_sharing_systems <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/bike_sharing_systems.csv"  
world_cities <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/world_cities.csv"

seoul_bike_sharing <- read.csv(seoul_bike_sharing, colClasses=c(DATE="character"))
cities_weather_forecast <- read.csv(cities_weather_forecast)
bike_sharing_systems <- read.csv(bike_sharing_systems)
world_cities <- read.csv(world_cities)

### Cast the Date column from `seoul_bike_sharing` to `Date`

Using the format "%d/%m/%Y", and check that the resulting datatypes are as expected.


In [ ]:
seoul_bike_sharing$DATE <- as.Date(seoul_bike_sharing$DATE, format = "%d/%m/%Y")
str(seoul_bike_sharing)

### Load the dataframes into the Db2 tables


In [ ]:
sqlSave(conn, seoul_bike_sharing, "SEOUL_BIKE_SHARING", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE,varTypes=c(DATE="Date"))
sqlSave(conn, cities_weather_forecast, "CITIES_WEATHER_FORECAST", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
sqlSave(conn, bike_sharing_systems, "BIKE_SHARING_SYSTEMS", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
sqlSave(conn, world_cities, "WORLD_CITIES", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)

### Check that the column `TYPE_NAME`s are appropriate in the resulting tables


In [ ]:
# e.g.:
sqlColumns(conn, "SEOUL_BIKE_SHARING")

Congratulations! Having successfully loaded your datasets into Db2, you are ready to move on to demonstrating your SQL querying skills. (Please close the connection)


In [ ]:
close(conn)

## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Rav Ahuja, Yan Luo </h4>

## Change log

| Date       | Version | Changed by    | Change Description                                                                                          |
| ---------- | ------- | ------------- | ----------------------------------------------------------------------------------------------------------- |
| 2021-04-22 | 0.2     | Jeff Grossman | Updated based on peer review                                                                                |
| 2021-04-05 | 0.1     | Jeff Grossman | Started content creation                                                                                    |

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
